In [10]:
from enum import Enum
import math
from queue import Empty
from typing import List, Optional


class Direction(Enum):
    NORTH = 'North'
    EAST = 'East'
    SOUTH = 'South'
    WEST = 'West'


class Axis(Enum):
    NORTH_SOUTH = 'North-South'
    EAST_WEST = 'East-West'


NORTH = Direction.NORTH
EAST = Direction.EAST
SOUTH = Direction.SOUTH
WEST = Direction.WEST
NORTH_SOUTH = Axis.NORTH_SOUTH
EAST_WEST = Axis.EAST_WEST


class Wall:
    def __init__(self, direction: Direction, axis: Axis, length: int, distance_to_me: int, distance_to_trainer: int):
        self.direction = direction
        self.axis = axis
        self.length = length
        self.distance_to_me = distance_to_me
        self.distance_to_trainer = distance_to_trainer

    @classmethod
    def BuildWall(cls, direction: Direction, dimensions, my_position, trainer_position):

        if direction == NORTH:
            length = dimensions[0]
            distance_to_me = dimensions[1] - my_position[1]
            distance_to_trainer = dimensions[1] - trainer_position[1]

        elif direction == EAST:
            length = dimensions[1]
            distance_to_me = dimensions[0] - my_position[0]
            distance_to_trainer = dimensions[0] - trainer_position[0]

        elif direction == SOUTH:
            length = dimensions[0]
            distance_to_me = my_position[1]
            distance_to_trainer = trainer_position[1]

        else:
            length = dimensions[1]
            distance_to_me = my_position[0]
            distance_to_trainer = trainer_position[0]

        return Wall(
            direction = direction,
            axis = NORTH_SOUTH if direction in [EAST, WEST] else EAST_WEST,
            length = length,
            distance_to_me = distance_to_me,
            distance_to_trainer= distance_to_trainer
        )


class Room:
    def __init__(self, dimensions, my_position, trainer_position):
        self.north = Wall.BuildWall(NORTH, dimensions, my_position, trainer_position)
        self.east = Wall.BuildWall(EAST, dimensions, my_position, trainer_position)
        self.south = Wall.BuildWall(SOUTH, dimensions, my_position, trainer_position)
        self.west = Wall.BuildWall(WEST, dimensions, my_position, trainer_position)
        self.dimensions = dimensions
        self.my_position = my_position
        self.trainer_position = trainer_position

    def walls(self):
        return [self.north, self.east, self.south, self.west]

    def fighter_distance(self):
        x = self.trainer_position[0] - self.my_position[0]
        y = self.trainer_position[1] - self.my_position[1]
        return math.sqrt(x ** 2 + y ** 2)


class EmptyPathException(Exception): pass


class ImpossibleTriangleException(Exception): pass


class Run:
    def __init__(self, num: int, walls: List[Wall], axis: Axis, turn_wall: Optional[Wall] = None):
        self.num = num
        self.walls = walls
        self.axis = axis
        self.turn_wall = turn_wall

    def first_wall(self):
        return self.walls[0]

    def current_wall(self):
        return self.walls[-1]

    def set_turn_wall(self, turn_wall: Wall):
        self.turn_wall = turn_wall


class Path:
    def __init__(self, room: Room, runs: Optional[List[Run]] = None):
        self.room = room
        self.runs = runs
    
    def copy(self):
        if self.runs is None: return Path(self.room)
        else: return Path(self.room, self.runs.copy())

    def first_run(self) -> Run:
        if self.runs is None: raise EmptyPathException
        else: return self.runs[0]

    def current_run(self):
        if self.runs is None: raise EmptyPathException
        else: return self.runs[-1]

    def first_wall(self):
        return self.first_run().first_wall()

    def current_wall(self):
        return self.current_run().current_wall()

    def with_updated_run(self, wall: Wall):
        path = self.copy()
        path.current_run().walls.append(wall)
        return path

    def with_new_run(self, wall: Wall):
        path = self.copy()

        if path.runs is None:
            path.runs = [Run(1, [wall], wall.axis)]

        else:
            run_num = len(path.runs) + 1
            
            if not path.current_run().turn_wall:
                path.current_run().set_turn_wall(wall)

            if path.current_wall() == self.room.north:
                turn_wall = self.room.south
            elif path.current_wall() == self.room.east:
                turn_wall = self.room.west
            elif path.current_wall() == self.room.south:
                turn_wall = self.room.north
            else:
                turn_wall = self.room.east

            path.runs.append(Run(run_num, [wall], wall.axis, turn_wall))

        return path

    def extended(self, wall: Wall):
        if self.runs is None or self.current_run().axis != wall.axis: return self.with_new_run(wall)
        else: return self.with_updated_run(wall)


    def calc_north_south_distance(self, room: Room):
        if self.runs is None:
            raise EmptyPathException

        distance = sum([room.dimensions[1] for run in self.runs if run.axis == NORTH_SOUTH])
        distance += sum([(len(run.walls) - 1) * room.dimensions[1] for run in self.runs if run.axis == EAST_WEST])
        distance += self.first_wall().distance_to_me if self.first_run().axis == EAST_WEST else 0
        distance += self.current_wall().distance_to_trainer if self.current_run().axis == EAST_WEST else 0
        
        if len(self.runs) == 1 and self.first_run().axis == NORTH_SOUTH:
            distance -= room.dimensions[1] - abs(room.my_position[1] - room.trainer_position[1])
        
        if len(self.runs) > 1:
            if self.first_run().turn_wall == room.south:
                distance -= room.north.distance_to_me
            elif self.first_run().turn_wall == room.north:
                distance -= room.south.distance_to_me
            
            if self.current_run().turn_wall == room.south:
                distance -= room.south.distance_to_me
            elif self.current_run().turn_wall == room.north:
                distance -= room.north.distance_to_me

        return distance


    def calc_east_west_distance(self, room: Room):
        if self.runs is None:
            raise EmptyPathException

        distance = sum([room.dimensions[0] for run in self.runs if run.axis == EAST_WEST])
        distance += sum([(len(run.walls) - 1) * room.dimensions[0] for run in self.runs if run.axis == NORTH_SOUTH])
        distance += self.first_wall().distance_to_me if self.first_run().axis == NORTH_SOUTH else 0
        distance += self.current_wall().distance_to_trainer if self.current_run().axis == NORTH_SOUTH else 0
        
        if len(self.runs) == 1 and self.first_run().axis == EAST_WEST:
            distance -= room.dimensions[0] - abs(room.my_position[0] - room.trainer_position[0])
        
        if len(self.runs) > 1:
            if self.first_run().turn_wall == room.west:
                distance -= room.east.distance_to_me
            elif self.first_run().turn_wall == room.east:
                distance -= room.west.distance_to_me
            
            if self.current_run().turn_wall == room.west:
                distance -= room.west.distance_to_me
            elif self.current_run().turn_wall == room.east:
                distance -= room.east.distance_to_me

        return distance


    def calc_beam_distance(self, room: Room):
        if self.runs is None: raise EmptyPathException

        total_north_south = self.calc_north_south_distance(room)
        total_east_west = self.calc_east_west_distance( room)
        first_run = self.runs[0]
        first_wall = first_run.walls[0]
        first_turn_wall = first_run.turn_wall
        opposite_axis = first_wall.axis

        if opposite_axis == NORTH_SOUTH:
            total_opposite = total_north_south
            total_adjacent = total_east_west
        else:
            total_opposite = total_east_west
            total_adjacent = total_north_south

        first_adjacent = first_wall.distance_to_me
        first_opposite = total_opposite / total_adjacent * first_adjacent

        if first_turn_wall and (first_opposite > first_turn_wall.distance_to_me):
            raise ImpossibleTriangleException

        first_hypotenuse = math.sqrt(first_adjacent ** 2 + first_opposite ** 2)
        return total_opposite / first_opposite * first_hypotenuse

In [11]:
def solution(dimensions, my_position, trainer_position, max_distance):

    ROOM = Room(dimensions, my_position, trainer_position)
    result = 0
    stack = [Path(ROOM)]

    while stack:

        path = stack.pop()
        
        try:
            if path.calc_beam_distance(ROOM) <= max_distance:
                result += 1
                stack.extend([path.extended(wall) for wall in ROOM.walls() if wall.direction != path.current_wall().direction])

        except EmptyPathException:
            if ROOM.fighter_distance() <= max_distance:
                result += 1
                stack.extend([path.extended(wall) for wall in ROOM.walls()])

        except ImpossibleTriangleException:
            continue
    
    return result

In [1]:
solution([3,2], [1,1], [2,1], 4)

NameError: name 'solution' is not defined